In [ ]:
from scipy.sparse import csr_matrix, load_npz, save_npz, coo_matrix
import numpy as np 
import pandas as pd 
from sklearn.metrics import roc_auc_score,average_precision_score
import matplotlib.pyplot as plt 
from scipy.stats import entropy
from tqdm import tqdm 
import os 
import json 
import types 
import glob 

import seaborn as sns

In [ ]:
from sparsechem import load_results

In [ ]:
pvs = ['0'
       ,'0_2'
       ,'0_4'
       ,'0_5'
       ,'0_6'
       ,'0_7'
       ,'0_8'
       ,'0_9'
       ,'0_95'
       ,'0_99'
      ]

In [ ]:
datapath = './aux_data_preperation/baseline_plus_aux_data/'
modelpath = '.'

In [ ]:
path = os.path.join(datapath, 'results_tmp/classification/T8c.csv')
t8c = pd.read_csv(path)

In [ ]:
sizes = [4000,6000,8000]
recs_baseline = []

for size in sizes : 
    path = modelpath + '/aux_data_training/baseline_{}/models/classification_baseline.json'.format(size)
    perf_baseline_aucroc = load_results(path)['validation']['classification_agg']['roc_auc_score']
    perf_baseline_aucpr = load_results(path)['validation']['classification_agg']['auc_pr']
    rec_baseline = pd.DataFrame({
                'AUC ROC':[perf_baseline_aucroc]
                ,'AUC PR':[perf_baseline_aucpr]
                ,'size':[size]
    })
    recs_baseline.append(rec_baseline)
    
out_baseline = pd.concat(recs_baseline) 

In [ ]:
ser_baselines = out_baseline.sort_values(by='AUC PR').iloc[-1]
ser_baselines

In [ ]:
for d in tqdm(glob.glob('./aux_data_training/ppv_npv_scan_{}_{}/*'.format(size,weight))):
    print(d)

In [ ]:

sizes = [4000,4000,4000,6000,8000]
weights = ['0_1','0_02','0_3','0_1','0_1']

p_roc = []
p_pr = []
p_spr = []
prim_perf = {}
prim_perf_baseline = {}

recs = []

    
perf_roc = 0 
perf_pr = 0
perf_spr = 0

for i,size in enumerate(sizes) : 
    
    weight = weights[i]
    
    for d in tqdm(glob.glob('./aux_data_training/ppv_npv_scan_{}_{}/*'.format(size,weight))): 
        path = os.path.join(d,'models','classification_baseline_w_aux.json')
        res = load_results(path)['validation']['classification_agg']

        thr = str(d).split('npv')[-1]

        perf_roc = res['roc_auc_score']       
        perf_pr = res['auc_pr']

        rec = pd.DataFrame({
                    'AUC ROC':[perf_roc]
                    ,'AUC PR':[perf_pr]
                    ,'quality_threshold':[thr]
                    ,'weight':[weight]
                    ,'size':[size]
        })

        recs.append(rec)

        
        
out = pd.concat(recs)

In [ ]:
df_res = out.groupby(['quality_threshold','weight','size']).agg([np.mean, np.min, np.max])

In [ ]:
metric = 'AUC ROC'

df_res_4000_01 = df_res.query('weight == "0_1" and size==4000')
df_res_4000_03 = df_res.query('weight == "0_3" and size==4000')
df_res_4000_002 = df_res.query('weight == "0_02" and size==4000')
df_res_6000_01 = df_res.query('weight == "0_1" and size==6000')
df_res_8000_01 = df_res.query('weight == "0_1" and size==8000')


_ = plt.plot(df_res_4000_01[metric]['mean'].values
            ,c='navy'
            ,ls='--')

_ = plt.plot(df_res_4000_03[metric]['mean'].values
            #,c='navy'
            ,ls='--')
_ = plt.plot(df_res_4000_002[metric]['mean'].values
            #,c='navy'
            ,ls='--')
_ = plt.plot(df_res_6000_01[metric]['mean'].values
            #,c='navy'
            ,ls='--')
_ = plt.plot(df_res_8000_01[metric]['mean'].values
            #,c='navy'
            ,ls='--')

plt.fill_between(
    list(range(len(pvs)))
    , df_res_4000_01[metric].amin.values
    , df_res_4000_01[metric].amax.values 
    ,color='lightblue'
)
_ = plt.xticks(range(len(pvs)),pvs)
plt.grid(axis='y'
        ,ls='--'
        )

plt.plot(range(len(pvs)),np.repeat(ser_baselines[metric],repeats=len(pvs)),c='red')

_ = plt.legend(
    [
        'layer size x1, weight 0.1'
        ,'layer size x1, weight 0.3'
        ,'layer size x1, weight 0.02'
        ,'layer size x1.5, weight 0.1'
        ,'layer size x2, weight 0.1'
    ]
)

In [ ]:
metric = 'AUC ROC'

df_res_4000_01 = df_res.query('weight == "0_1" and size==4000')
df_res_4000_03 = df_res.query('weight == "0_3" and size==4000')
df_res_4000_002 = df_res.query('weight == "0_02" and size==4000')
df_res_6000_01 = df_res.query('weight == "0_1" and size==6000')
df_res_8000_01 = df_res.query('weight == "0_1" and size==8000')


_ = plt.plot(df_res_4000_01[metric]['mean'].values-ser_baselines[metric]
            ,c='blue'
            ,ls='--')

_ = plt.plot(df_res_4000_03[metric]['mean'].values-ser_baselines[metric]
            ,c='green'
            ,ls='--')
_ = plt.plot(df_res_4000_002[metric]['mean'].values-ser_baselines[metric]
            ,c='gray'
            ,ls='--')
_ = plt.plot(df_res_6000_01[metric]['mean'].values-ser_baselines[metric]
            ,c='orange'
            ,ls='--')
_ = plt.plot(df_res_8000_01[metric]['mean'].values-ser_baselines[metric]
            ,c='brown'
            ,ls='--')

plt.fill_between(
    list(range(len(pvs)))
    , df_res_4000_01[metric].amin.values-ser_baselines[metric]
    , df_res_4000_01[metric].amax.values-ser_baselines[metric]
    ,color='lightblue'
)
_ = plt.xticks(range(len(pvs)),pvs)
plt.grid(axis='y'
        ,ls='--')

plt.plot(range(len(pvs)),np.repeat(0,repeats=len(pvs)), c='red')

_ = plt.legend(
    [
        'layer size x1, weight 0.1'
        ,'layer size x1, weight 0.3'
        ,'layer size x1, weight 0.02'
        ,'layer size x1.5, weight 0.1'
        ,'layer size x2, weight 0.1'
        ,'baseline'
    ]
)

plt.ylim([-.004,.004])

_ = plt.ylabel(metric)
_ = plt.xlabel('task quality criterium (NPV and PPV > threshold)\nused for task selection\n(training)')

plt.savefig('./results/{}_reruns.png'.format(metric)
                      ,bbox_inches='tight'
            ,pad_inches=0.2)

In [ ]:
metric = 'AUC PR'

df_res_4000_01 = df_res.query('weight == "0_1" and size==4000')
df_res_4000_03 = df_res.query('weight == "0_3" and size==4000')
df_res_4000_002 = df_res.query('weight == "0_02" and size==4000')
df_res_6000_01 = df_res.query('weight == "0_1" and size==6000')
df_res_8000_01 = df_res.query('weight == "0_1" and size==8000')


_ = plt.plot(df_res_4000_01[metric]['mean'].values-ser_baselines[metric]
            ,c='blue'
            ,ls='--')

_ = plt.plot(df_res_4000_03[metric]['mean'].values-ser_baselines[metric]
            ,c='green'
            ,ls='--')
_ = plt.plot(df_res_4000_002[metric]['mean'].values-ser_baselines[metric]
            ,c='gray'
            ,ls='--')
_ = plt.plot(df_res_6000_01[metric]['mean'].values-ser_baselines[metric]
            ,c='orange'
            ,ls='--')
_ = plt.plot(df_res_8000_01[metric]['mean'].values-ser_baselines[metric]
            ,c='brown'
            ,ls='--')


plt.fill_between(
    list(range(len(pvs)))
    , df_res_4000_01[metric].amin.values-ser_baselines[metric]
    , df_res_4000_01[metric].amax.values-ser_baselines[metric]
    ,color='lightblue'
)
_ = plt.xticks(range(len(pvs)),pvs)
plt.grid(axis='y'
        ,ls='--')

plt.plot(range(len(pvs)),np.repeat(0,repeats=len(pvs)), c='red')

_ = plt.legend(
    [
        'layer size x1, weight 0.1'
        ,'layer size x1, weight 0.3'
        ,'layer size x1, weight 0.02'
        ,'layer size x1.5, weight 0.1'
        ,'layer size x2, weight 0.1'
        ,'baseline'
    ]
)

plt.ylim([-.004,.004])

_ = plt.ylabel(metric)
_ = plt.xlabel('task quality criterium (NPV and PPV > threshold)\nused for task selection\n(training)')

plt.savefig('./results/{}_reruns.png'.format(metric)
                      ,bbox_inches='tight'
            ,pad_inches=0.2)